<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignmnet 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [ ]:
from tensorflow import keras 
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

import numpy as np

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [ ]:
x_train.shape

In [ ]:
x_train[:13]

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
boston_model = Sequential()

boston_model.add(Dense(24, input_dim=13, activation='relu'))
boston_model.add(Dense(12, activation='relu'))
boston_model.add(Dense(12, activation='relu'))
boston_model.add(Dense(1))
boston_model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [ ]:
from tensorflow import keras
from keras.utils import plot_model
import pydot
import matplotlib.pyplot as plt

In [ ]:
history = boston_model.fit(x_train,
                    y_train,
                    validation_split=0.20,
                    epochs=20,
                    batch_size=16,
                    verbose=0)

In [ ]:
history.history['mean_squared_error']

In [ ]:
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - MSE')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - MSE')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
y_test = y_test.reshape(-1,1)

In [ ]:
y_test.shape

In [ ]:
pred = pred.reshape(-1,1)

In [ ]:
pred.shape

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print(mean_squared_error(pred, y_test))

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [ ]:
from tensorflow.keras.datasets import mnist
import tensorflow

In [ ]:
batch_size = 50
num_classes = 10
epochs = 15

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
mnist_model = Sequential()

#input
mnist_model.add(Dense(16, input_dim=784, activation='relu'))
# hidden
mnist_model.add(Dense(16, activation='relu'))
mnist_model.add(Dense(16, activation='relu'))
# output 
mnist_model.add(Dense(10, activation='softmax'))
# compile
mnist_model.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

mnist_model.summary()

In [ ]:
history = mnist_model.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_split=.1, verbose=0)
scores = mnist_model.evaluate(x_test, y_test)
print(f'{mnist_model.metrics_names[1]}: {scores[1]*100}')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model accuracy')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.